In [1]:
#skeleton code from : https://www.kaggle.com/code/abanobmorgan/vgg19-classifierimport numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import Subset, ConcatDataset , DataLoader
from collections import OrderedDict
import torch.optim as optim
from sklearn.metrics import confusion_matrix

In [2]:
for dirname, _, filenames in os.walk('raw-img'):
    for filename in filenames:
        path, folder = os.path.split(dirname)
        # print(path,folder)

In [3]:
train_on_gpu = torch.cuda.is_available()
data_transform1 = transforms.Compose([transforms.RandomRotation(45),
                                     transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(1080),
                                     transforms.Resize(512),
                                     transforms.Resize(224),
                                     transforms.RandomRotation(45),
                                      transforms.ToTensor()])

data_transform2 = transforms.Compose([transforms.RandomHorizontalFlip(),
                                     transforms.RandomResizedCrop(1080),
                                     transforms.Resize(224),
                                     transforms.RandomRotation(45),
                                      transforms.RandomRotation(35),
                                      transforms.ToTensor()])

In [4]:
dataset1 = datasets.ImageFolder(path,transform=data_transform1)
dataset2 = datasets.ImageFolder(path,transform=data_transform2)

In [5]:
maxlen=750
for l, cls in enumerate(dataset1.classes):
    if l == 0 :
        idx = [i for i in range(len(dataset1) ) if dataset1.imgs[i][1] == dataset1.class_to_idx[dataset1.classes[l]]]
        subset = Subset(dataset1, idx)
        master= Subset(subset,idx [:maxlen])
        subset = Subset(dataset2, idx[:maxlen])
        master= ConcatDataset((master, subset))
        
        print(len(master))
    else : 
        idx = [i for i in range(len(dataset1) ) if dataset1.imgs[i][1] == dataset1.class_to_idx[dataset1.classes[l]]]
        
        subset = Subset(dataset1, idx[:maxlen])
        master= ConcatDataset((master, subset))
        subset = Subset(dataset2, idx[:maxlen])
        master= ConcatDataset((master, subset))
        print(len(master))
        #print(len(master))
valid_size = 0.1
test_size = 0.1
num_train = len(master)
indices = list(range(num_train))
np.random.shuffle(indices)
valid_split = int(np.floor((valid_size) * num_train))
test_split = int(np.floor((valid_size+test_size) * num_train))
valid_idx, test_idx, train_idx = indices[:valid_split], indices[valid_split:test_split], indices[test_split:]

1500
3000
4500


In [6]:
num_workers = 6
batch_size= 60
disimage = 20

In [7]:
train_loader = Subset(master, train_idx)
valid_loader = Subset(master,valid_idx )
test_loader = Subset(master,test_idx )

train_loader =torch.utils.data.DataLoader(train_loader, batch_size=batch_size, num_workers=num_workers)
valid_loader =torch.utils.data.DataLoader(valid_loader, batch_size=batch_size, num_workers=num_workers)
test_loader =torch.utils.data.DataLoader(test_loader, batch_size=batch_size, num_workers=num_workers)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
classes = ["cane","gallina","elefante"]
translate = {"cane": "dog","elefante": "elephant","gallina": "chicken"}

In [10]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

In [11]:
model = models.vgg19(pretrained=True)

classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 6000)),
                                         ('relu', nn.ReLU()),
                                         ('dropout', nn.Dropout(.5)), 
                                         ('fc2', nn.Linear(6000, 10)), 
                                         ('output', nn.Softmax(dim=1) )]))                                  

model.classifier = classifier

print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [12]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=0.001)
num_classes = len(classes)
def seq (model, df, name ): 
    train_loss = 0.0
    class_correct = list(0. for i in range(num_classes))
    class_total = list(0. for i in range(num_classes))
    for batch_i, (data, target) in enumerate(df):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            model.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        if name == 'train': 
            loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss 
        train_loss += loss.item()
        _, pred = torch.max(output, 1) 
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        for i in range(len(target.data)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        
    return class_correct, class_total, train_loss

In [13]:
def printdata(class_correct, class_total, train_loss, epoch, name, df ): 
    print(f'Epoch %d, loss: %.8f \t{name} Accuracy (Overall): %2d%% (%2d/%2d)' %(epoch,
        train_loss / len(df), 100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    if ((epoch+1) % 5 == 0 or epoch == 1):
        for i in range(num_classes):
            if class_total[i] > 0:
                print(f'{name} Accuracy of %5s: %2d%% (%2d/%2d)' % (
                translate[classes[i]], 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))

In [14]:
def trainModel (model, train_loader,valid_loader, num_epochs=20): 
    # number of epochs to train the model
    n_epochs = num_epochs
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        class_correct = list(0. for i in range(num_classes))
        class_total = list(0. for i in range(num_classes))
        ###################
        # train the model #
        ###################
       # Repeat for each batch in the training set
        model.train()
        class_correct, class_total, train_loss= seq(model,  train_loader, 'train')
        printdata(class_correct, class_total, train_loss, epoch, 'train', train_loader)
        # Repeat for each validation batch 
        ###################
        # validate the model #
        ###################
        model.eval()
        class_correct, class_total, train_loss= seq(model, valid_loader, 'validation')
        printdata(class_correct, class_total, train_loss, epoch, 'validation', valid_loader)
    torch.save(model.state_dict(), 'model.pt')        
    print(f'model saved ')

In [15]:
print(len(train_loader), len(valid_loader))
trainModel (model, train_loader,valid_loader,20)
 # track test loss 
# over 10 animals classes
test_loss = 0.0
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))
model.eval()
class_correct, class_total, train_loss= seq(model, test_loader, 'test')
printdata(class_correct, class_total, train_loss, 1, 'test', test_loader)
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

60 8
Epoch 1, loss: 2.26128888 	train Accuracy (Overall): 30% (1088/3600)
train Accuracy of   dog: 31% (374/1202)
train Accuracy of chicken: 23% (281/1198)
train Accuracy of elephant: 36% (433/1200)
Epoch 1, loss: 2.20704770 	validation Accuracy (Overall): 38% (175/450)
validation Accuracy of   dog: 28% (43/150)
validation Accuracy of chicken:  7% (11/153)
validation Accuracy of elephant: 82% (121/147)
Epoch 2, loss: 2.14042170 	train Accuracy (Overall): 40% (1460/3600)
Epoch 2, loss: 2.08428624 	validation Accuracy (Overall): 41% (187/450)
Epoch 3, loss: 2.03641042 	train Accuracy (Overall): 51% (1837/3600)
Epoch 3, loss: 1.99523239 	validation Accuracy (Overall): 61% (277/450)
Epoch 4, loss: 1.92907165 	train Accuracy (Overall): 63% (2285/3600)
train Accuracy of   dog: 63% (764/1202)
train Accuracy of chicken: 58% (703/1198)
train Accuracy of elephant: 68% (818/1200)
Epoch 4, loss: 1.89018868 	validation Accuracy (Overall): 63% (285/450)
validation Accuracy of   dog: 60% (90/150)
val

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0.